<a href="https://colab.research.google.com/github/ShatilKhan/7-Days-of-LangChain/blob/main/Demo_of_Simulated_Legal_Voicebot_with_Private_RAG_Backend_and_Zoho_CRM_Integration.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Setup and Library Installation

First, install necessary libraries and initialize the language model and vector database. We use Hugging Face Transformers and ChromaDB for the RAG backend.

In [ ]:
!pip install transformers chromadb sentence_transformers
# Note: Installing libraries might take a minute.
import chromadb
from transformers import pipeline
from sentence_transformers import SentenceTransformer


Knowledge Base Ingestion and Vector Store Initialization

Next, we simulate ingesting a private legal document into an in-memory vector store (ChromaDB). We create a couple of mock legal documents (for example, one about landlord-tenant law and another about contract law) to serve as our knowledge base. These documents will be embedded into vectors using a pre-trained model and stored in ChromaDB for efficient similarity search

In [ ]:
# Prepare mock legal documents
doc1_text = """Landlord-Tenant Law:
If a tenant does not pay rent, the landlord can serve a notice to pay or quit. If the tenant still fails to pay,
the landlord may initiate eviction proceedings according to local housing laws. Tenants have the right to respond
to an eviction notice in court. For serious lease violations or illegal behavior, an unconditional quit notice may be served."""

doc2_text = """Contract Law:
When one party breaches a contract, the other party may seek remedies such as damages or specific performance.
A material breach allows the non-breaching party to terminate the contract and sue for damages. Contracts require
offer, acceptance, and consideration to be valid. Certain contracts must be in writing under the Statute of Frauds."""

# Initialize embedding model (MiniLM for embeddings)
embed_model = SentenceTransformer('all-MiniLM-L6-v2')  # Downloads model if not already available

# Initialize ChromaDB in-memory client and collection
client = chromadb.Client()
collection = client.create_collection(name="legal_knowledge")

# Embed the documents and add them to the Chroma collection
docs = [doc1_text, doc2_text]
doc_ids = ["doc1", "doc2"]
embeddings = embed_model.encode(docs).tolist()  # Convert NumPy array to list for Chroma

collection.add(documents=docs, ids=doc_ids, embeddings=embeddings)
print(f"Indexed {len(docs)} documents in the vector store: {doc_ids}")


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Indexed 2 documents in the vector store: ['doc1', 'doc2']


Retrieval-Augmented QA Pipeline Setup

With the documents indexed, we set up a retrieval function and a QA pipeline. The retrieval function will query the Chroma vector store for relevant document snippets given a user question. We then feed the question and retrieved context into an open-source QA model to simulate the LLM's answer generation. In a real system, this could be a larger language model (e.g., Mistral-7B or Llama 2) generating a detailed answer from the context. For this demo, we use a smaller pre-trained question-answering model for simplicity

In [ ]:
# Define a retrieval function using the vector store
def retrieve_context(question, top_k=1):
    # Embed the question using the same model
    q_emb = embed_model.encode([question]).tolist()
    # Query Chroma for similar documents (most relevant context)
    results = collection.query(query_embeddings=q_emb, n_results=top_k)
    if results.get('documents'):
        # Join retrieved documents into one context string
        context = "\n".join(results['documents'][0])
    else:
        context = ""
    return context

# Initialize a question-answering pipeline (using a smaller model for demo; can use larger LLM in practice)
qa_pipeline = pipeline("question-answering", model="deepset/roberta-base-squad2")

# Test the RAG pipeline with a sample question
sample_question = "What can I do if my tenant refuses to pay rent?"
print(f"User question: {sample_question}")
# Retrieve relevant context from the knowledge base
context = retrieve_context(sample_question, top_k=1)
print("Retrieved context:\n", context)
# Generate an answer using the QA model (LLM)
if context:
    result = qa_pipeline({"question": sample_question, "context": context})
    print("LLM Answer:", result['answer'])
else:
    print("LLM Answer: [No relevant information found in knowledge base]")


config.json:   0%|          | 0.00/571 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/496M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/79.0 [00:00<?, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/772 [00:00<?, ?B/s]

Device set to use cpu
/usr/local/lib/python3.12/dist-packages/transformers/pipelines/question_answering.py:395: FutureWarning: Passing a list of SQuAD examples to the pipeline is deprecated and will be removed in v5. Inputs should be passed using the `question` and `context` keyword arguments instead.
  warnings.warn(


User question: What can I do if my tenant refuses to pay rent?
Retrieved context:
 Landlord-Tenant Law:
If a tenant does not pay rent, the landlord can serve a notice to pay or quit. If the tenant still fails to pay, 
the landlord may initiate eviction proceedings according to local housing laws. Tenants have the right to respond 
to an eviction notice in court. For serious lease violations or illegal behavior, an unconditional quit notice may be served.
LLM Answer: serve a notice to pay or quit


Simulating the Voice Call Flow with State Machine

Now that the backend QA system is ready, we simulate a phone call conversation using a simple state machine to manage the flow. The state machine will handle:

Call initiation: Simulate an incoming call via Twilio.

Greeting: The bot greets the user and prompts for their question.

Listening: Simulate capturing the user's question via ASR (Automatic Speech Recognition).

Answering: Retrieve and generate the answer using our RAG backend.

Responding: Output the answer via TTS (Text-to-Speech) simulation.

Lead Qualification: Decide if the call should be treated as a sales lead (e.g., the user has a legal issue that warrants follow-up).

CRM Integration: If qualified, create a lead in Zoho CRM, update its status, and trigger an outbound follow-up call.

Call termination: End the call after the interaction.

We use placeholder functions to simulate ASR and TTS. Twilio call events (like starting a call or receiving audio) are represented by print statements to mimic real-time WebSocket events. The state machine logic logs each step of the conversation and integration clearly.

In [ ]:
# Placeholder functions for ASR and TTS simulation
def simulate_asr(audio_stream_data):
    """
    Simulate ASR by returning transcribed text for given audio data.
    In this demo, we ignore the actual audio and return a preset example question.
    """
    transcribed_text = "What can I do if my tenant refuses to pay rent?"
    print(f"[Simulated ASR]: {transcribed_text}")
    return transcribed_text

def simulate_tts(text):
    """
    Simulate TTS by 'speaking' the text.
    Here we just print the text to represent the voicebot's spoken response.
    """
    print(f"[Bot Voice]: {text}")

# Placeholder functions for Twilio and CRM interactions
def twilio_start_call(from_number):
    print(f"Twilio Event: Incoming call from {from_number}. Starting voice stream...")

def twilio_end_call():
    print("Twilio Event: Call has ended.")

def twilio_outbound_call(to_number):
    print(f"Twilio Event: Initiating outbound call to {to_number} for follow-up.")

def zoho_create_lead(lead_data):
    print(f"Zoho CRM API: Creating new lead with data: {lead_data}")
    # Simulate API response with a new Lead ID
    response = {"status": "success", "lead_id": "LEAD12345"}
    print(f"Zoho CRM API Response: {{'status': '{response['status']}', 'lead_id': '{response['lead_id']}'}}")
    return response["lead_id"]

def zoho_update_lead(lead_id, update_data):
    print(f"Zoho CRM API: Updating lead {lead_id} with data: {update_data}")
    # Simulate API response for the update
    print(f"Zoho CRM API Response: {{'status': 'success', 'updated_fields': {list(update_data.keys())}}}")

# Simulate an incoming call and conversation flow
caller_number = "+1-202-555-0136"  # Example caller ID
twilio_start_call(caller_number)

# State machine for call flow
state = "greeting"
lead_id = None
user_question = None

while True:
    if state == "greeting":
        # Bot greets the user
        simulate_tts("Hello, thank you for calling LegalAssist. How can I help you today?")
        # Move to listening state to get the user's question
        state = "listening"

    elif state == "listening":
        # Simulate receiving audio stream from user and transcribing it via ASR
        user_question = simulate_asr(audio_stream_data=None)
        # Log the captured question text
        print(f"User question captured: {user_question}")
        # Proceed to answer the question
        state = "answering"

    elif state == "answering":
        # Use RAG backend to retrieve context and generate an answer
        print("Retrieving relevant information from knowledge base...")
        context = retrieve_context(user_question, top_k=1)
        if context:
            print("Context found. Generating answer with LLM...")
            result = qa_pipeline({"question": user_question, "context": context})
            bot_answer = result['answer']
        else:
            bot_answer = "I'm sorry, I don't have information on that topic."
        # Respond to the user via TTS
        simulate_tts(bot_answer)
        # After answering, move to lead qualification
        state = "lead_qualification"

    elif state == "lead_qualification":
        # Simple lead qualification logic:
        # If the user's question contains certain keywords (e.g., 'tenant'), treat it as a qualified lead.
        if user_question and "tenant" in user_question.lower():
            print("Lead Qualification: User's issue likely requires follow-up. Marking as qualified lead.")
            # Create a new lead in Zoho CRM
            lead_data = {"Name": "Unknown", "Phone": caller_number, "Query": user_question}
            lead_id = zoho_create_lead(lead_data)
            # Update the lead status after the call
            zoho_update_lead(lead_id, {"Status": "Called", "CallOutcome": "Answered by bot"})
            # Trigger an outbound call to follow up (e.g., connecting to a human agent or scheduling a consultation)
            twilio_outbound_call(caller_number)
        else:
            print("Lead Qualification: Issue did not meet criteria for a new lead. No follow-up will be triggered.")
        # End the call after qualification step
        state = "call_end"

    elif state == "call_end":
        simulate_tts("Thank you for using LegalAssist. Goodbye!")
        twilio_end_call()
        break


Twilio Event: Incoming call from +1-202-555-0136. Starting voice stream...
[Bot Voice]: Hello, thank you for calling LegalAssist. How can I help you today?
[Simulated ASR]: What can I do if my tenant refuses to pay rent?
User question captured: What can I do if my tenant refuses to pay rent?
Retrieving relevant information from knowledge base...
Context found. Generating answer with LLM...
[Bot Voice]: serve a notice to pay or quit
Lead Qualification: User's issue likely requires follow-up. Marking as qualified lead.
Zoho CRM API: Creating new lead with data: {'Name': 'Unknown', 'Phone': '+1-202-555-0136', 'Query': 'What can I do if my tenant refuses to pay rent?'}
Zoho CRM API Response: {'status': 'success', 'lead_id': 'LEAD12345'}
Zoho CRM API: Updating lead LEAD12345 with data: {'Status': 'Called', 'CallOutcome': 'Answered by bot'}
Zoho CRM API Response: {'status': 'success', 'updated_fields': ['Status', 'CallOutcome']}
Twilio Event: Initiating outbound call to +1-202-555-0136 for f